In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import time
%matplotlib inline
#import evolution
from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
vc = VirtualCoach(environment='local')

INFO: [2018-02-04 01:01:34,254 - rospy.topics] topicmanager initialized


INFO: [2018-02-04 01:01:34,404 - Configuration] Loading configuration file config.json
INFO: [2018-02-04 01:01:34,407 - Configuration] Using user specified environment: local
INFO: [2018-02-04 01:01:34,466 - VirtualCoach] Ready.


## Helper Functions
Some helper functions to calculate the fitness function, plot the robot's trajectory and the wheel speeds. These functions are specific to this experiment.

In [2]:
def fitness_function(wheel_speeds):
    left_wheel = ([float(t[1]) for t in wheel_speeds[11:]])
    right_wheel = ([float(t[2]) for t in wheel_speeds[11:]])
    fitness = 0
    for i in range(len(left_wheel)):
        if left_wheel[i] >= 0 and right_wheel[i] >= 0:
                   fitness += (left_wheel[i] + right_wheel[i])
    return fitness/float(2*len(left_wheel))

def get_top_performers(population, fitness_log, num_performers=10):
    """
    Extract the indices of the top individuals from the fitness log.
    
    :param fitness_log: fitness function scores for all individuals in a population
    :param num_performers: number for top performers to look for. Default value is
                           15, which corresponds to a truncation threshold of 25% in
                           this experiment.
    """
    top_performers = []
    for i in range(num_performers):
        max_index = np.argmax(fitness_log)
        print max_index
        top_performers.append(population[max_index])
        fitness_log[max_index] = -1
        
        file = open("top_performers.txt","w") 
        file.write(str(top_performers))
        file.write("\n")
        file.close() 


    return top_performers

def plot_trajectory(trajectory):
    plt.figure()
    plt.gca()
    plt.xticks([], [])
    plt.yticks([], [])
    plt.ylim(-3, 3)
    plt.xlim(-3.9, 3.9)
    x_axis = [x[0] for x in trajectory[2:]]
    y_axis = [y[1] for y in trajectory[2:]]
    plt.plot([float(x) for x in x_axis], [float(y) for y in y_axis])

def plot_wheel_speeds(wheel_speeds):
    left_wheel = ([float(t[1]) for t in wheel_speeds[11:]])
    right_wheel = ([float(t[2]) for t in wheel_speeds[11:]])
    plt.plot(range(len(left_wheel)), left_wheel, 'b')
    plt.plot(range(len(right_wheel)), right_wheel, 'r')

Evolutionary Algorithm helper functions

In [3]:
def one_point_crossover(parent1, parent2):
        parent1 = parent1.reshape(290)
        parent2 = parent2.reshape(290)
        child1 = np.zeros(290, dtype=int)
        child2 = np.zeros(290, dtype=int)
        point = np.random.randint(len(parent1))
        for i in range(point):
            child1[i] = parent1[i]
            child2[i] = parent2[i]
        for i in range(point, 290):
            child1[i] = parent2[i]
            child2[i] = parent1[i]
        child1 = child1.reshape(10, 29)
        child2 = child2.reshape(10, 29)
        return child1, child2
    
def bit_mutation(population):
        for individual in population:
            individual = individual.reshape(290)
            for i in range(290):
                if np.random.rand() < 0.05:
                    individual[i] = 0 if individual[i] else 1
        return population
    
def get_unique_pairs(population):
    pairs = []
    for i in range(len(population)):
        for j in range(i+1, len(population)):
            pairs.append((i, j))
    return pairs
            
def evolve_new_generation(top_performers):
    population = []
    for i in range(len(top_performers)):
        for j in range(4):
            population.append(top_performers[i])
    pairs = get_unique_pairs(population)
    for i in pairs:
        if np.random.rand() < 0.1:
            parent1 = population[i[0]]
            parent2 = population[i[1] - 1]
            child1, child2 = one_point_crossover(parent1, parent2)
            population[i[0]] = child1
            population[i[1]] = child2

    population = bit_mutation(population)
    rand = np.random.randint(len(population))
    population[rand] = top_performers[0]
    
    file = open("populations.txt","w") 
    file.write(str(population))
    file.write("\n")
    file.close() 
        
    return population

## The Brain
The PyNN script that creates the neural network stored as a string. A new binary genetic string that encodes the connections between neurons is passed on each run.

In [4]:
brain = """from hbp_nrp_cle.brainsim import simulator as sim
import numpy as np
import logging

logger = logging.getLogger(__name__)

dna = np.array([int(x) for x in '%s'.split(',')]).reshape(10, 29)

receptors = []
for r in range(1,19):
    receptors.append(np.nonzero(dna[:,r])[0])


def create_brain():

    NEURONPARAMS = {'v_rest': -60.5,
                    'tau_m': 4.0,
                    'tau_refrac': 2.0,
                    'tau_syn_E': 10.0,
                    'tau_syn_I': 10.0,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_thresh': -60.4,
                    'v_reset': -60.5}

    SYNAPSE_PARAMS = {"weight": 1.0,
                      "delay": 2.0}

    population = sim.Population(10, sim.IF_cond_alpha())
    population[0:10].set(**NEURONPARAMS)


    # Connect neurons
    CIRCUIT = population

    SYN = sim.StaticSynapse(**SYNAPSE_PARAMS)

    row_counter=0
    for row in dna:
    	logger.info(row)
        n = np.array(row)
        r_type = 'excitatory'
        if n[0]==0:
            r_type = 'inhibitory'
        for i in range(19,29):
            if n[i]==1:
                sim.Projection(presynaptic_population=CIRCUIT[row_counter:1+row_counter], postsynaptic_population=CIRCUIT[i-19:i-18], connector=sim.OneToOneConnector(), synapse_type=SYN, receptor_type=r_type)
        
        row_counter+=1

    sim.initialize(population, v=population.get('v_rest'))

    logger.debug("Circuit description: " + str(population.describe()))

    return population


circuit = create_brain()

"""

In [5]:
display_trial_tf = """@nrp.Robot2Neuron()
def display_trial_number(t):
    clientLogger.advertise('%s')
"""

## Run Experiment

In [6]:
class FloreanoExperiment(object):
    
    def __init__(self, population, generations):
        self.last_status = [None]
        self.population = population
        self.fitness_log = []
        self.sim = None
        self.started = False
        self.generations = generations
        self.sim_data = []

    def wait_condition(self, timeout, condition):
        start = time.time()
        while time.time() < start + timeout:
            time.sleep(0.25)
            if condition(self.last_status[0]):
                return
        raise Exception('Condition check failed')
        
    def on_status(self, status):
        self.last_status[0] = status
                 
    def save_simulation_data(self, trial):
        self.sim_data[trial].append([])
        wheel_speeds = self.sim.get_csv_data('wheel_speeds.csv')
        try:
            left_wheel = ([float(t[1]) for t in wheel_speeds[11:]])
        except Exception as inst:
            print(type(inst))
            print(inst.args)
            print(inst)
            for t in wheel_speeds:
                print(t)
        right_wheel = ([float(t[2]) for t in wheel_speeds[11:]])
        trajectory = self.sim.get_csv_data('robot_position.csv')
        fitness = fitness_function(wheel_speeds)
        self.sim_data[trial][-1] = {
            'fitness': fitness,
            'wheel_speeds': wheel_speeds,
            'left_wheel': left_wheel,
            'right_wheel': right_wheel,
            'trajectory': trajectory
        }

    def run_experiment(self):
        try:
            self.sim = vc.launch_experiment('floreano')
        except:
            time.sleep(1)
        self.sim.register_status_callback(self.on_status)
        for i in range(self.generations):
            self.sim_data.append([])
            for j in range(40):
                print("Generation {}, Population {}".format(i, j))
                genetic_string = ','.join(str(x) for x in population[j].ravel())
                self.sim.edit_brain(brain % genetic_string)
                self.sim.add_transfer_function(display_trial_tf % "Generation {}, Population {}".format(i, j) )
                self.sim.start()
                # run simulation for 40 seconds
                self.wait_condition(1000, lambda x: x['simulationTime'] > 40)
                self.sim.pause()
                self.save_simulation_data(i)
                self.sim.reset('full')
                self.wait_condition(100, lambda x: x['state'] == 'paused' and x['simulationTime'] == 0)
            self.fitness_log = [result['fitness'] for result in floreano_experiment.sim_data[i]]
            self.top_performers = get_top_performers(self.population, list(self.fitness_log))
            self.population = evolve_new_generation(self.top_performers)
            file = open("data_dump.txt","w") 
            file.write(str(floreano_experiment.sim_data))
            file.write("\n")
            file.close() 

In [ ]:
population = np.random.randint(2, size=(40, 10, 29)) # random population of 10 binary genetic strings
floreano_experiment = FloreanoExperiment(population, 40)
floreano_experiment.run_experiment()

INFO: [2018-02-04 01:01:51,633 - VirtualCoach] Preparing to launch floreano.
INFO: [2018-02-04 01:01:51,634 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-02-04 01:01:51,656 - VirtualCoach] Retrieving list of available servers.
[{u'gzweb': {u'assets': u'http://localhost:8080/assets',
             u'nrp-services': u'http://localhost:8080',
             u'videoStreaming': u'http://localhost:8080/webstream/',
             u'websocket': u'ws://localhost:8080/gzbridge'},
  u'id': u'localhost',
  u'rosbridge': {u'websocket': u'ws://localhost:8080/rosbridge'},
  u'serverJobLocation': u'local'}]
INFO: [2018-02-04 01:01:51,661 - Simulation] Attempting to launch floreano on localhost.
INFO: [2018-02-04 01:01:59,918 - Simulation (floreano - localhost #0)] Simulation Successfully Created.
INFO: [2018-02-04 01:01:59,931 - Simulation (floreano - localhost #0)] Ready.
INFO: [2018-02-04 01:01:59,932 - Simulation (floreano - localhost #0)] Status callback registered.
Generation 0, Populati

INFO: [2018-02-04 01:03:35,804 - Simulation (floreano - localhost #0)] Attempting to set Transfer Function 
INFO: [2018-02-04 01:03:35,814 - Simulation (floreano - localhost #0)] Transfer Function '' successfully updated
INFO: [2018-02-04 01:03:35,814 - Simulation (floreano - localhost #0)] Attempting to transition to state: started
INFO: [2018-02-04 01:03:35,820 - Simulation (floreano - localhost #0)] Simulation state: started
INFO: [2018-02-04 01:04:18,857 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2018-02-04 01:04:18,861 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-02-04 01:04:18,868 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-04 01:04:18,889 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-04 01:04:18,908 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-04

INFO: [2018-02-04 01:05:57,829 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: [2018-02-04 01:05:58,031 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: plane
INFO: [2018-02-04 01:05:58,251 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the brain
INFO: [2018-02-04 01:05:58,747 - Simulation (floreano - localhost #0)] Reset completed. The simulation has been paused and will not be startedautomatically.
Generation 0, Population 5
INFO: [2018-02-04 01:05:59,749 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-04 01:05:59,761 - Simulation (floreano - localhost #0)] Attempting to set Brain
INFO: [2018-02-04 01:05:59,762 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-04 01:05:59,767 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-04 01:05:59,772 - Simulation (floreano - localhost #0)]

INFO: [2018-02-04 01:08:15,311 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-02-04 01:08:15,312 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-04 01:08:15,400 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-04 01:08:15,418 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-04 01:08:15,435 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-04 01:08:15,452 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2018-02-04 01:08:15,455 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-02-04 01:08:15,456 - Simulation (floreano - localhost #0)] Attempting to reset full
INFO: [2018-02-04 01:08:16,399 - Simulation (floreano - localhost #0)] [Resetting the simulation] 
INFO: [2018-02-04 01:08:16,400 - Simulation 

Generation 0, Population 10
INFO: [2018-02-04 01:09:49,457 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-04 01:09:49,470 - Simulation (floreano - localhost #0)] Attempting to set Brain
INFO: [2018-02-04 01:09:49,470 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-04 01:09:49,476 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-04 01:09:49,480 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-04 01:09:49,485 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-04 01:09:49,962 - Simulation (floreano - localhost #0)] Brain successfully updated.
INFO: [2018-02-04 01:09:49,963 - Simulation (floreano - localhost #0)] Attempting to retrieve transfer-function
INFO: [2018-02-04 01:09:49,968 - Simulation (floreano - localhost #0)] Attempting to set Transfer Function 
INFO: [2018-02-04 01:09:49,976 - Simulation (floreano - lo

...

INFO: [2018-02-05 12:13:11,639 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-05 12:13:11,674 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2018-02-05 12:13:11,680 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-02-05 12:13:11,681 - Simulation (floreano - localhost #0)] Attempting to reset full
INFO: [2018-02-05 12:13:24,484 - Simulation (floreano - localhost #0)] [Resetting the simulation] 
INFO: [2018-02-05 12:13:24,486 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the 3D world
INFO: [2018-02-05 12:13:25,249 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: pointlight_0
INFO: [2018-02-05 12:13:25,664 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: plane
INFO: [2018-02-05 12:13:25,968 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: 

INFO: [2018-02-05 12:25:25,876 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-05 12:25:25,880 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-05 12:25:26,297 - Simulation (floreano - localhost #0)] Brain successfully updated.
INFO: [2018-02-05 12:25:26,298 - Simulation (floreano - localhost #0)] Attempting to retrieve transfer-function
INFO: [2018-02-05 12:25:26,303 - Simulation (floreano - localhost #0)] Attempting to set Transfer Function 
INFO: [2018-02-05 12:25:26,314 - Simulation (floreano - localhost #0)] Transfer Function '' successfully updated
INFO: [2018-02-05 12:25:26,314 - Simulation (floreano - localhost #0)] Attempting to transition to state: started
INFO: [2018-02-05 12:25:26,318 - Simulation (floreano - localhost #0)] Simulation state: started
INFO: [2018-02-05 12:31:11,938 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2018-02-05 12:31:11,943 - Simulation 

INFO: [2018-02-05 12:43:27,461 - Simulation (floreano - localhost #0)] [Resetting the simulation] 
INFO: [2018-02-05 12:43:27,461 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the 3D world
INFO: [2018-02-05 12:43:27,465 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: pointlight_0
INFO: [2018-02-05 12:43:27,878 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: plane
INFO: [2018-02-05 12:43:28,215 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: [2018-02-05 12:43:28,420 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: plane
INFO: [2018-02-05 12:43:28,656 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the brain
INFO: [2018-02-05 12:43:29,085 - Simulation (floreano - localhost #0)] Reset completed. The simulation has been paused and will not be startedautomatically.
Generation 24, Population 2
INFO: [20

INFO: [2018-02-05 12:55:26,088 - Simulation (floreano - localhost #0)] Attempting to set Transfer Function 
INFO: [2018-02-05 12:55:26,099 - Simulation (floreano - localhost #0)] Transfer Function '' successfully updated
INFO: [2018-02-05 12:55:26,100 - Simulation (floreano - localhost #0)] Attempting to transition to state: started
INFO: [2018-02-05 12:55:26,106 - Simulation (floreano - localhost #0)] Simulation state: started
INFO: [2018-02-05 13:01:15,228 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2018-02-05 13:01:15,251 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-02-05 13:01:15,252 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-05 13:01:15,294 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-05 13:01:15,332 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-02-05

INFO: [2018-02-05 13:13:44,930 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: [2018-02-05 13:13:45,135 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: plane
INFO: [2018-02-05 13:13:45,370 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the brain
INFO: [2018-02-05 13:13:45,772 - Simulation (floreano - localhost #0)] Reset completed. The simulation has been paused and will not be startedautomatically.
Generation 24, Population 7
INFO: [2018-02-05 13:13:46,524 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-05 13:13:46,529 - Simulation (floreano - localhost #0)] Attempting to set Brain
INFO: [2018-02-05 13:13:46,529 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-05 13:13:46,537 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-02-05 13:13:46,542 - Simulation (floreano - localhost #0)

In [8]:
fig, axes = plt.subplots(len(floreano_experiment.sim_data), 2)
for i in range(len(floreano_experiment.sim_data)):
    axes[i, 0].set_ylim(-3, 3)
    axes[i, 0].set_xlim(-3.9, 3.9)
    axes[i, 0].set_xticks([], [])
    axes[i, 0].set_yticks([], [])
    x_axis = [x[0] for x in floreano_experiment.sim_data[i]['trajectory'][2:]]
    y_axis = [y[1] for y in floreano_experiment.sim_data[i]['trajectory'][2:]]
    axes[i, 0].plot([float(x) for x in x_axis], [float(y) for y in y_axis])

    left_wheel = ([float(t[1]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][2:]])
    right_wheel = ([float(t[2]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][2:]])
    axes[i, 1].plot(range(len(left_wheel)), left_wheel, 'b', label='Left Wheel')
    axes[i, 1].plot(range(len(right_wheel)), right_wheel, 'r', label='Right Wheel')
    axes[i, 1].set_ylim(-5, 5)
    axes[i, 1].set_xlabel('Time [ms]')
    axes[i, 1].set_ylabel('Speed m/s')

axes[0, 0].set_title('Robot Trajectory')
axes[0, 1].set_title('Wheel Speeds')
fig.set_figheight(25)
fig.set_figwidth(10)


TypeError: expected a string or other character buffer object